In [ ]:
import pandas as pd
fn = '../src/data/raw/Trees_data.csv'
tree_df = pd.read_csv(fn)
# tree_df.sum()

In [ ]:
tree_df.head().sum()

In [ ]:
genus_list = pd.Series([blah[0] for blah in tree_df['Species'].str.split()]).unique()
tree_df['Genus'] = [blah[0] for blah in tree_df['Species'].str.split()]
tree_df['Genus'].unique()

In [ ]:
tree_df.iloc[0:2792,:].to_csv('Trees_genus.csv', index=False)

In [ ]:
#gb = tree_df.groupby(list(tree_df['Genus'].values))
tree_df['idx'] = tree_df.groupby('Genus').count()
# keep_list = ['Inventory date', 'Active']
# tree_df.pivot(index='Genus',columns='idx')[keep_list]
tree_df['idx']